In [1]:
import pandas as pd
import numpy as np
import holoviews as hv

from functools import wraps
import logging

def log_shape(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        logging.info("%s,%s" % (func.__name__, result.shape))
        return result
    return wrapper

def log_dtypes(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        logging.info("%s,%s" % (func.__name__, result.dtypes))
        return result
    return wrapper

In [4]:
prod_df = pd.read_csv('./billa_shop_products.csv')
prod_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10123 entries, 0 to 10122
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   articleId  10123 non-null  object 
 1   name       10123 non-null  object 
 2   price      10123 non-null  float64
 3   brand      9549 non-null   object 
 4   group      10123 non-null  object 
dtypes: float64(1), object(4)
memory usage: 395.6+ KB


In [8]:
prod_df['group'].unique()

array(['Obst & Gemüse', 'Brot & Gebäck', 'Getränke', 'Kühlwaren',
       'Tiefkühl', 'Grundnahrungsmittel', 'Süßes & Salziges', 'Pflege',
       'Haushalt', 'Haustier', 'Geschenkideen'], dtype=object)

In [10]:
prod_df.groupby('group').size().sort_values(ascending=False)

group
Grundnahrungsmittel    2659
Kühlwaren              1789
Getränke               1680
Süßes & Salziges        905
Tiefkühl                738
Pflege                  641
Haushalt                605
Brot & Gebäck           435
Haustier                340
Obst & Gemüse           278
Geschenkideen            53
dtype: int64

In [13]:
prod_df.groupby('group')['price'].describe().sort_values('mean', ascending=False)

,count,mean,std,min,25%,50%,75%,max
group,,,,,,,,
Geschenkideen,53.0,16.649811,16.743175,1.89,4.29,8.49,21.99,61.99
Getränke,1680.0,6.148911,8.109358,0.27,1.59,3.29,7.49,61.99
Kühlwaren,1789.0,5.395394,7.409653,0.29,1.59,2.59,4.19,66.99
Tiefkühl,738.0,4.810976,2.997192,0.69,3.29,3.99,5.49,19.99
Pflege,641.0,4.772059,3.629718,0.49,2.59,3.89,5.49,29.99
Haushalt,605.0,4.311140,3.095568,0.59,1.99,3.49,5.49,24.99
Haustier,340.0,3.245088,3.594358,0.20,0.99,1.99,4.49,24.99
Brot & Gebäck,435.0,3.048276,2.724431,0.19,1.59,2.29,3.49,21.90
Obst & Gemüse,278.0,2.791978,1.456861,0.69,1.99,2.49,2.99,10.99


---

# Prod SQLite database

In [ ]:
import sqlite3

con = sqlite3.connect('./crawler/data/products_billa.sqlite3')

with con:
    query_reduced_products = '''
        SELECT prod_id, MIN(price_normal), MAX(price_normal)
        FROM prices
        GROUP BY prod_id
        HAVING (MAX(price_normal) - MIN(price_normal)) < 0
        '''
    res = con.execute(query_reduced_products).fetchall()
res

## Price difference analysis

In [ ]:
-- price_changes_weekly (
--    prod_id,
--    week,
--    price_change_from_prev_week
--)

SELECT
    week,
    category,
    AVG(closing_price_last_week) AS avg_closing_price_last_week,
    AVG(closing_price_this_week) AS avg_closing_price_this_week,
    AVG(price_change_from_prev_week / closing_price_last_week) AS  avg_relative_price_change,
    AVG(price_change_from_prev_week) AS average_price_change,
    COUNT(*) as products
FROM price_changes_weekly JOIN products ON prod_id = id
[[ WHERE attributes LIKE '%'||{{product_attribute}}||'%' ]]
GROUP BY week, category
ORDER BY WEEK ASC, category ASC

In [ ]:
-- VIEW
-- price_changes_from_last_recorded
--    prod_id,
--    timestamp,
--    week,
--    price_now,
--    price_last_recorded,
--    price_change_from_last_recorded


SELECT
    category,
    name,
    id,
    price_first AS first_price,
    price_last AS last_price,
    price_change,
    date(timestamp_first, 'unixepoch') AS 'from',
    date(timestamp_last, 'unixepoch') AS 'to'
FROM
    (SELECT
        DISTINCT prod_id,
        ( FIRST_VALUE(price_now) OVER (product_window) ) AS price_first,
        ( LAST_VALUE(price_now) OVER (product_window) ) AS price_last,
        ( SUM(price_change_from_last_recorded) OVER (product_window) ) AS price_change,
        ( FIRST_VALUE(timestamp) OVER (product_window) ) AS timestamp_first,
        ( LAST_VALUE(timestamp) OVER (product_window) ) AS timestamp_last,
        ROW_NUMBER() OVER (PARTITION BY prod_id ORDER BY timestamp DESC) AS row_number
    FROM price_changes_from_last_recorded
    WINDOW product_window AS (PARTITION BY prod_id ORDER BY timestamp ASC))
    JOIN products ON prod_id = id
WHERE row_number = 1 AND price_change != 0 [[ AND name LIKE '%'||{{product_name}}||'%' ]] [[ AND {{product_category}} ]] 
ORDER BY price_change DESC

In [ ]:
-- VIEW
-- price_changes_from_last_recorded
--    prod_id,
--    timestamp,
--    week,
--    price_now,
--    price_last_recorded,
--    price_change_from_last_recorded

SELECT
    date(timestamp, 'unixepoch') AS 'date',
    category,
    name,
    prod_id,
    grammage,
    price_last_recorded AS price_previous,
    price_now,
    price_change_from_last_recorded AS price_change,
    rank
FROM price_changes_from_last_recorded
     JOIN products ON prod_id = id
WHERE price_change != 0 [[ AND category = {{category}} ]]
ORDER BY date DESC, CAST(rank AS INTEGER) DESC

In [12]:
import sqlite3
import holoviews as hv
import pandas as pd

con = sqlite3.connect('../../crawl-infra/database/products_billa.sqlite3')
with con:
    query = '''
        WITH weekly_price_changes AS (
            SELECT
                prod_id,
                week,
                price_last_recorded,
                SUM(price_change_from_last_recorded) AS price_change_from_prev_week,
                SUM(price_change_from_last_recorded) / price_last_recorded AS relative_price_change_from_prev_week
            FROM price_changes_from_last_recorded
            GROUP BY prod_id, week)
        
        SELECT
            category,
            week,
            AVG(relative_price_change_from_prev_week) AS avg_relative_price_change_from_prev_week,
            AVG(price_change_from_prev_week) AS avg_price_change_from_prev_week
        FROM weekly_price_changes JOIN products ON prod_id = id
        GROUP BY category, week
    '''
    res = con.execute(query).fetchall()
    
res

[('Brot & Gebäck', 18, 0.0, 0.0),
 ('Brot & Gebäck', 19, 0.0013681379231021337, 0.0031377551020408193),
 ('Brot & Gebäck', 20, 0.004161876515402895, 0.012984693877551015),
 ('Brot & Gebäck', 21, 0.006699629960261696, 0.02257731958762886),
 ('Geschenkideen', 18, 0.0, 0.0),
 ('Geschenkideen', 19, 0.0, 0.0),
 ('Geschenkideen', 20, 0.0, 0.0),
 ('Geschenkideen', 21, 0.016952744225471512, 0.0727272727272728),
 ('Getränke', 18, 0.0, 0.0),
 ('Getränke', 19, 6.55305165220327e-05, 0.0003640776699029124),
 ('Getränke', 20, 0.006705873085376722, 0.0216269113149847),
 ('Getränke', 21, 0.007638269018499608, 0.035359116022099485),
 ('Grundnahrungsmittel', 18, 0.0, 0.0),
 ('Grundnahrungsmittel', 19, 0.000820999607281893, 0.0013109381401065138),
 ('Grundnahrungsmittel', 20, 0.0073456311085601765, 0.016729585556011485),
 ('Grundnahrungsmittel', 21, 0.016306350846789025, 0.03726630660573328),
 ('Haushalt', 18, 0.0, 0.0),
 ('Haushalt', 19, 0.0, 0.0),
 ('Haushalt', 20, 0.0019373297172120285, 0.014907872696